In [1]:
# Importing the required libraries
import mediapipe as mp
import numpy as np
import cv2

In [2]:
# Defining a custom class containing all the relevant functions pertaining to the
# hand tracking functionality
class Hand_Tracker:
    
#     Constructor for the class
    def __init__(self, mode = False, maximum_hands = 2, detection_confidence = 0.5,
             tracking_confidence = 0.5):
        
#         Defining the relevant parameters for creating the hand detector object using mediapipe
        self.mode = mode
        self.maximum_hands = maximum_hands
        self.detection_confidence = detection_confidence
        self.tracking_confidence = tracking_confidence
        
        self.hand_detector = mp.solutions.hands.Hands(self.mode, self.maximum_hands,
                                                      self.detection_confidence,
                                                      self.tracking_confidence)
        
        
#         Initializing the list of position of hand landmarks at each frame
        self.landmark_lists = []
        
#         Initializing the list of tip Id's of each finger tip
        self.tip_ids = [4, 8, 12, 16, 20]
        
#     Helper function to find the list of positions of hand landmarks in the given frame
    def find_hand_landmarks(self, image, draw = True):
        
#         Setting the colour code of the input image to RGB
        image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#         Using the mediapipe library to determine the hand landmark positions
        self.hand_locations = self.hand_detector.process(image_RGB)
        
#         Graphically visualize the hand landmarks if required
        if(draw == True):
            if(self.hand_locations.multi_hand_landmarks != None):
                for hand_landmarks in self.hand_locations.multi_hand_landmarks:
                    
#                     Visualize the hand landmarks
                    mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks,
                                                              mp.solutions.hands.HAND_CONNECTIONS)
                    
        return image
    
#     Helper function to get the list of positions of each hand landmark in respect to their position
#     on the screen
    def find_hand_positions(self, image, hand_number = 0, draw = True):
        
#         Initializing the resultant list
        landmark_list = []
    
#         if the hand locations list in not empty
        if(self.hand_locations.multi_hand_landmarks != None):
        
#             get the location of the hand as desired (0 = first hand, 1 = second hand and so on)
            hand_in_image = self.hand_locations.multi_hand_landmarks[hand_number]
    
#             iterating the list of hand landmarks and processing only the important landmarks
            for landmark_id, landmark in enumerate(hand_in_image.landmark):
        
#                 Getting the height, width and the number of channels of the image
                image_height, image_width, image_channels = image.shape
    
#                 Getting the absolute position of the landmark on the screen and appending it 
#                 to the resultant list
                landmark_list.append([landmark_id, int(image_width * landmark.x),
                                           int(image_height * landmark.y)])

#         Appending the processed list to the list of hand landmark positions
        self.landmark_lists.append(landmark_list)
    
        return landmark_list
    
#     Helper function to determine if the given finger is up in the given image
    def is_finger_up(self, finger_id, landmark_list):
        
#         Return false if the landmark list is empty ie there is no hand in the image
        if(len(landmark_list) == 0 or len(landmark_list[finger_id]) == 0):
            return False
        
#         If the tip of the finger is higher than the tip of the thumb
        if(landmark_list[finger_id][2] < landmark_list[4][2]):
            return True
        else:
            return False
    
#     Helper function to determine the mode of the hand (If only index finger is up - Drawing mode
#     If index finger and middle finger are up - Selection mode
#     Else None mode
    def find_hand_mode(self):
        
#         Get the latest hand landmarks positions list
        latest_landmark_list = self.landmark_lists[-1]
    
#         If the index finger is up
        finger_up_index_finger = self.is_finger_up(8, latest_landmark_list)
    
#         If the middle finger is up
        finger_up_middle_finger = self.is_finger_up(12, latest_landmark_list)
    
#         If both index finger and middle finger are up
        if((finger_up_index_finger == True) and (finger_up_middle_finger == True)):
            return 'selection_mode'
        
#         If only index finger is up
        elif(finger_up_index_finger == True):
            return 'drawing_mode'
        
#         Else return None
        else:
            return None   

In [3]:
# Setting the source for image reading
source = cv2.VideoCapture(0)

# Setting the width and height of the image respectively
source.set(3, 1280)
source.set(4, 720)

# Creating a Hand_Tracker object
hand_tracker = Hand_Tracker()

In [4]:
# Initializing the previous and current positions of the tip of the finger
previous_location = (0, 0)
current_location = (0, 0)

# Initializing the previous hand mode
previous_hand_mode = None

# Initializing the black canvas for drawing
black_canvas = np.zeros((720, 1280, 3), np.uint8)

# Defining the BGR values for various colours
RED = (0, 0, 255)
BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
CANVAS_COLOUR = (0, 0, 0)

# Initializing the current paint colour
current_paint_colour = RED

# Setting the tracking circle radius
tracking_circle_radius = 40

# Setting the painting brush thickness
brush_thickness = 20

# Setting the eraser thickness
eraser_thickness = 35

In [5]:
# Defining the keyboard to colour mapping dictionary
# This dictionary is used to map the keyboard buttons to the various colours
key_colour_mapping = {}
key_colour_mapping[ord('r')] = RED
key_colour_mapping[ord('b')] = BLUE
key_colour_mapping[ord('g')] = GREEN
key_colour_mapping[ord('e')] = CANVAS_COLOUR

In [6]:
# Running the loop to execute main functionality of the AI virtual painter
while(True):
    
#     Taking the input from the keyboard
    key = cv2.waitKey(1)
    
#     If the escape key is pressed, end the application
    if(key == 27):
        break
        
#     If the keyboard input corresponds to one of the colour options present then change the
#     colour accordingly
    if(key in key_colour_mapping):
        current_paint_colour = key_colour_mapping[key]
    
#     Read the live image from the source
    read_success, live_image = source.read()
    
#     Horizontally flip the image so that the movement of the finger corresponds to the movement
#     of the finger on the screen
    live_image = cv2.flip(live_image, 1)
    
#     Draw the hand landmarks on the live image
    live_image = hand_tracker.find_hand_landmarks(live_image)
    
#     Find the hand landmarks from the input live image     
    hand_landmarks_list = hand_tracker.find_hand_positions(live_image)
    
#     Determine the hand mode from the live image
    current_hand_mode = hand_tracker.find_hand_mode()
    
#     If the current hand mode is not None
    if(current_hand_mode != None):
        
#         Getting the location of the tip of the index finger of the hand
        current_location = (hand_landmarks_list[8][1], hand_landmarks_list[8][2])
    
#         Setting the finger tracker colour according to the paint colour
        tracker_colour = current_paint_colour
    
#         Draw the tracking circle 
        cv2.circle(live_image, current_location, tracking_circle_radius, tracker_colour)
    
#         If the hand is in the drawing mode
        if(current_hand_mode == 'drawing_mode'):
            
#             If the hand has come to the drawing mode for the first (This condition is applied
#             so that no extraneous lines are drawn)  
            if(previous_location == (0, 0) or previous_hand_mode != 'drawing_mode'):
                previous_location = current_location
                
#             If the painter does not want to erase the content
            if(current_paint_colour != CANVAS_COLOUR):
        
#                 Draw the content drawn by the painter
                cv2.line(black_canvas, previous_location, current_location, current_paint_colour,
                     brush_thickness)
                
            else:
#                 Erase the content accordingly
                cv2.line(black_canvas, previous_location, current_location, current_paint_colour,
                     eraser_thickness)
            
#     Updating the previous location of the tip of the index finger for calculations
#     for the next frame
    previous_location = current_location
    
#     Updating the previous hand mode of the tip of the index finger for calculations
#     for the next frame
    previous_hand_mode = current_hand_mode
    
#     Display the live image to track the hand of the painter
    cv2.imshow('LIVE', live_image)
    
#     Display the drawing canvas
    cv2.imshow('Canvas', black_canvas)

In [7]:
# Releasing the image source after the completion of the application
cv2.destroyAllWindows()
source.release()